In [13]:
from enum import Enum
precision = 6
class LanguageLvl(Enum):
    A1 = 0
    A2 = 1
    B1 = 2
    B2 = 3
    C1 = 4
    C2 = 5

class Language:
    lvl: LanguageLvl
    name: str
    id: list[int]

    def __init__(self, level: int, lang_name: str, lang_id: list):
        self.lvl = level
        self.name = lang_name
        self.id = lang_id

In [9]:
class Topic:
    id: list[int]
    name: str

In [10]:
from enum import Enum
class StudentState(Enum):
    Initial = 0
    Working = 1
    Learning = 2
    Inactive = 3
    Dead = 4

class Qualification(Enum):
    Low = 0
    Middle = 1
    High = 2

In [30]:
import numpy as np
def calcNextState(vec) -> StudentState:
    random_num = np.random.uniform(size=1)[0]
    bottom_border = 0.0
    for i in range(len(vec)):
        if bottom_border <= random_num <= sum(vec[:i + 1]):
            return i + 1
        else:
            bottom_border += vec[i]

In [ ]:
import numpy as np
import typing
import scipy.stats as sps
from collections.abc import Callable

class Student:
    id: int
    state: StudentState
    fatige: np.double
    learning_langs: list[Language]
    native_language: Language
    average_time_per_task: list[tuple(Language, np.double)]
    lang_qualification: list[tuple(Language, Qualification)]
    activity_times: list[tuple(Topic, np.datetime64)]
    motivated: bool
    initial_test_result: np.double
    __memory_threshold_prob: np.double
    __memory_threshold_max_prob: np.double
    __forgetting_min_prob: np.double
    __forgetting_prob: list[tuple(Topic, np.double)]
    __memorized_topics: list[Topic]
    __hard_topics: list[Topic]
    _seed: int
    _distribution_params: dict
    _activity_distribution_function: Callable[[int, dict], list[np.datetime64]]
    _next_event_time: np.datetime64
    __activity_tmstmp: np.array
    __activity_tmstmp_count: int

    def __init__(self, ID: int, learning_langs: list[Language], native_lang: Language, 
    average_task_time: list[tuple(Language, np.double)], initial_qualification: list[tuple(Language, Qualification)],
    max_prob_threshold: np.double, forgetting_normal_prob: np.double, registration_date: np.datetime64, 
                 initial_test: np.double, motivated: bool):
        self.id = ID
        self.state = StudentState.Initial
        self.fatige = 0.0
        self.native_language = native_lang
        self.learning_langs = learning_langs
        self.average_time_per_task = average_task_time
        self.lang_qualification = initial_qualification
        self.__memory_theshold_max_prob = max_prob_threshold
        self.__memory_threshold_prob = self.__memory_theshold_max_prob
        self.__forgetting_min_prob = forgetting_normal_prob
        self._next_event_time = registration_date
        self.initial_test_result =  initial_test
        self.motivated = motivated
        
        self.forgetting_prob = []
        self.__memorized_topics = []
        self.__hard_topics = []
        
    def setWorkingDates(start_date: np.datetime64) -> np.array[np.datetime64]:
        # TODO

    def solveSomeTasks() -> np.array[tuple(Task, np.datetime64)]:
        # TODO
    
    def call() -> np.datetime64:
        # initial state processing
        if self.state == StudentState.Initial:
            self.state = calcNextState([0.6, 0.3, 0.08, 0.02])
            if self.state == StudentState.Inactive:
                self._next_event_time = np.random.uniform(low = self._next_event_time, high = self._next_event_time \
                                                          + np.timedelta64(100, 'D'))
            elif self.state == StudentState.Dead:
                self._next_event_time = np.datetime64('1970-01-01T00:00')

            elif self.state == StudentState.Learning:
                self._next_event_time = np.random.uniform(low = self._next_event_time, high = self._next_event_time \
                                                          + np.timedelta64(10, 'D'))
                # TODO: implements the increasing of positive probabililty
                increasePositiveProb()
            
            elif self.state == StudentState.Working:
                self.setWorkingDates(self._next_event_time)
                self._next_event_time = self.__activity_tmstmp[0]
                self.__activity_tmstmp_count = 0
        
        # current state is working
        elif self.state == StudentState.Working:
            working_duration = self.calcWorkDuration()
            self.solveSomeTasks()
            self._next_event_time += working_duration
            prob_i = np.array([
                0.0,                             #W 
                np.around(1.0 / 3.0, precision), #L
                np.around(1.0 / 3.0, precision), #I
                np.around(1.0 / 3.0, precision), #D
            ])
            langs = getUniqueLangs()
            result_prob = np.array([0.0, 0.0, 0.0, 0.0])         
            vectors = np.array([prob_i for _ in len(langs)])
            for i, language in enumerate(langs):
                if language.lvl < LanguageLvl.B1:
                    vectors[i][2] += 0.1
                    vectors[i][1] -= 0.1
                if self.motivated:
                    vectors[i][1] += 0.2
                    vectors[i][2] -= 0.1
                    vectors[i][3] -= 0.1
                if self.__activity_tmstmp[self.__activity_tmstmp_count + 1] \
                - self.__activity_tmstmp[self.__activity_tmstmp_count] <= np.timedelta64(24, 'H'):
                    vectors[i][1] -= 0.05
                    vectors[i][2] += 0.05
                if self.__activity_tmstmp[self.__activity_tmstmp_count + 1] \
                - self.__activity_tmstmp[self.__activity_tmstmp_count] > np.timedelta64(24, 'H'):
                    vectors[i][1] += 0.03
                    vectors[i][2] -= 0.03
                for j in len(vectors[i]):
                    result_prob[j] += vectors[i][j]

            for i in range(len(result_prob)):
                result_prob[i] = result_prob[i] / np.double(len(langs))

            self.state = calcNextState(result_prob)
            if self.state == StudentState.Dead:
                self._next_event_time = np.datetime64('1970-01-01T00:00')
            
            self.__activity_tmstmp_count += 1
        
        # other states
        elif self.state != StudentState.Dead:
            if self.state == StudentState.Learning:
                # TODO: implements the increasing of positive probabililty
                increasePositiveProb()
            elif self.state == StudentState.Inactive:
                # TODO: implements the decreasing of positive probability
                decreasePositiveProb()
            self.state = calcNextState([1.0, 0.0, 0.0, 0.0])
            if self.state == StudentState.Working:
                self._next_event_time = self.__activity_tmstmp[self.__activity_tmstmp_count]'
        
        return self._next_event_time